# H.04 | Linear Regression

We'll revisit our penguin friends for H.04. Over the course of this homework assignment, you will be asked to train a regression model to predict the `body_mass_g` of a penguin using their `bill_length_mm`, `bill_depth_mm`, and `flipper_length_mm` as features.

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import plotly.express as px

# Read in data.
df = pd.read_csv("https://storage.googleapis.com/mbai-data/body_weight_regression.csv")
df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,39.1,18.7,181.0,3750.0
1,39.5,17.4,186.0,3800.0
2,40.3,18.0,195.0,3250.0
3,36.7,19.3,193.0,3450.0
4,39.3,20.6,190.0,3650.0


## Create $X$ and $y$

Recall that in supervised learning, we have a dataset consisting of both input features and output labels. The goal is to learn a model that can predict the target labels (y) from the input features (X).

By convention, we can place our target variable in the "rightmost" column of our dataset. Below, we create an X matrix that contains both the training data (X) and labels (y).

In [26]:
# Create a feature matrix X. The feature at column -1 is our target (body_mass_g).
X = df.values

# Print X Shape (214, 4) -- 3 features and 1 target.
print("Shape: ", X.shape)

# Print (X)
print("X (First 3 columns) & y (last column, body_mass_g): \n", X[:5])

Shape:  (214, 4)
X (First 3 columns) & y (last column, body_mass_g): 
 [[  39.1   18.7  181.  3750. ]
 [  39.5   17.4  186.  3800. ]
 [  40.3   18.   195.  3250. ]
 [  36.7   19.3  193.  3450. ]
 [  39.3   20.6  190.  3650. ]]


## Split the Data

Now let's split our data into a training set and a testing set. We will not use a validation set in this homework assignment, because we don't need to tune any hyperparameters. We will use the training set to train our models and the testing set to evaluate our models.

Please write a function `split_data` using the instructions in regression.py.

In [27]:
from regression import split_data

train, test = split_data(X, shuffle = False)

In [28]:
print("Train shape (171 training samples, 3 features + 1 target): ", train.shape)
print("Test shape (43 testing samples, 3 features + 1 target): ", test.shape)

Train shape (171 training samples, 3 features + 1 target):  (171, 4)
Test shape (43 testing samples, 3 features + 1 target):  (43, 4)


## Standardize x_train and x_test.

Please implement standard scaling to standardize the feature dataframe. Recall that standard scaling is defined as:

$$ x_{\text{standardized}} = \frac{x - \mu}{\sigma} $$

where $\mu$ is the mean of the feature and $\sigma$ is the standard deviation of the feature. Please write a function `standardize_train_and_test_data` that takes in training and testing data and returns the standardized training and testing data. You can use the `StandardScaler` class from Scikit-Learn to accomplish this. 

The `fit_transform` method will fit the scaler to the training data and transform it, while the `transform` method will transform the testing data using the same scaler.

**NOTE**: Make sure to standardize the training and testing data separately. Do not standardize the entire dataset at once, as this would leak information from the testing set into the training set. You should fit the scaler to the training data and then use that same scaler to transform the testing data. This will result in the column means / stds for the training set will be 0 / 1 respectively, while the testing set should be *near* 0 / 1 respectively. 

In [ ]:
from regression import standardize

train, test = standardize(train, test)

print("Train shape (171 training samples, 3 features + 1 target): ", train.shape)
print("Train per-column means: ", np.mean(train, axis = 0).round(2))
print("Train per-column stds: ", np.std(train, axis = 0).round(2))

print("--------")

print("Test shape (43 testing samples, 3 features + 1 target): ", test.shape)
print("Test per-column means: ", np.mean(test, axis = 0).round(2))
print("Test per-column stds: ", np.std(test, axis = 0).round(2))

Train shape (171 training samples, 3 features + 1 target):  (171, 4)
Train per-column means:  [-0. -0. -0.  0.]
Train per-column stds:  [1. 1. 1. 1.]
--------
Test shape (43 testing samples, 3 features + 1 target):  (43, 4)
Test per-column means:  [1.97 0.03 1.16 0.15]
Test per-column stds:  [0.73 1.01 1.06 0.94]


## Linear Regression

Now that we have our scaled training and testing sets, let's implement our linear regression model. We will use linear regression to predict the body_mass_g of a given penguin given their other features.

You will be asked to implement the following functions:

1. `linear_regression`

Recall the equation for the normal equation:

$$ \theta = (X^T X)^{-1} X^T y $$

You need to add a bias term in this function. The bias term is a column of ones that is added to the input feature matrix $X$ and allows the model to learn an intercept. The normal equation / the output of this function will return the optimal linear regression slopes (aka the 'm' in $mx + b$) and the bias term (the 'b' in $mx + b$).

2. `linear_regression_predict`

Recall the equation for linear regression:

$$ \hat{y} = X \theta $$

In other words, we will use the slopes / bias term from the previous function to make predictions on the test set. The output of this function will be the predicted values for the test set.

3. `mean_squared_error`

Recall the equation for the mean squared error:
$$ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$

Please see more details in regression.py. You may only use the numpy library. You **may not** use the scikit-learn library.

### Single Feature Performance

In our first pass of the model, let's quantify our performance when using only the **flipper_length_mm** feature.

In [42]:
from regression import linear_regression, linear_regression_predict, mean_squared_error


flipper_length_x_train = train[:, -2] # Select the flipper_length_mm feature (train)
body_mass_y_train = train[:, -1] # Select the target body_mass_g (train)

flipper_length_x_test = test[:, -2] # Select the flipper_length_mm feature (test)
body_mass_y_test = test[:, -1] # Select the target body_mass_g (test)

# Calculate the linear regression weights using the training data.
linear_regression_weights = linear_regression(flipper_length_x_train.reshape(-1, 1), train[:, -1])

# Calculate the predicted body mass using the linear regression weights.
body_mass_pred = linear_regression_predict(flipper_length_x_test.reshape(-1, 1), linear_regression_weights)

# Calculate the mean squared error of the predictions.
mse = mean_squared_error(test[:, -1], body_mass_pred)

print("Mean Sqaured Error (predicting body_mass_g using only flipper_length_mm):", round(mse, 2))

Mean Sqaured Error (predicting body_mass_g using only flipper_length_mm): 0.61


### Visualization of Single Feature Performance

It is always a good idea to visualize our results. You don't have to do anything here. You can see that our model is doing a fine job of finding the best possible slope when only using one feature, but there is a lot of unnaccounted-for variance between the true data and our predictions. We'll use the rest of our features in the second pass.

In [39]:
# plot demonstrating the linear regression model.
fig = px.scatter(x=flipper_length_x_train, y=body_mass_y_train, template = "plotly_white", title = "Regression")

# plot the regression line.
x_line = np.linspace(flipper_length_x_train.min(), flipper_length_x_train.max(), 100)
y_line = linear_regression_weights[1] * x_line + linear_regression_weights[0]
fig.add_scatter(x=x_line, y=y_line, mode="lines", name="Regression Line", line=dict(color="red", dash="dash"))

# plot each test sample.
fig.add_scatter(x=flipper_length_x_test, y=body_mass_y_test, mode="markers", name="Test Samples", marker=dict(size=10, color="grey"))

# plot each predicted test sample.
fig.add_scatter(x=flipper_length_x_test, y=body_mass_pred, mode="markers", name="Predicted Test Samples", marker=dict(size=10, color="red"))

# Update axes names.
fig.update_layout(
    xaxis_title="Flipper Length (mm)",
    yaxis_title="Body Mass (g)",
    showlegend=True,
)

### All Feature Performance

Our first pass wasn't so bad! But we can probably do better. Now, we'll include all of the remaining features (bill_length_mm, bill_depth_mm) in our model. You will see a lower mean squared error when using all the features in our simple linear regression model.

In [ ]:
from regression import linear_regression, linear_regression_predict, mean_squared_error

x_train = train[:, :-1] # Select the flipper_length_mm feature (train)
y_train = train[:, -1] # Select the target body_mass_g (train)

x_test = test[:, :-1] # Select the flipper_length_mm feature (train)
y_test = test[:, -1] # Select the target body_mass_g (train)

# Calculate the linear regression weights using the training data.
linear_regression_weights = linear_regression(x_train, y_train)

# Calculate the predicted body mass using the linear regression weights.
body_mass_pred = linear_regression_predict(x_test, linear_regression_weights)

# Calculate the mean squared error of the predictions.
mse = mean_squared_error(test[:, -1], body_mass_pred)
print("Mean Squared Error (predicting body_mass_g using all features):", round(mse, 2))

Mean Squared Error ((predicting body_mass_g using all features): 0.48
